In [1]:
from bhmm.hidden.api import forward

In [2]:
from pyemma.msm import MaximumLikelihoodMSM, MaximumLikelihoodHMSM
from msmbuilder.cluster import NDGrid
from sklearn.pipeline import Pipeline
import pickle
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# traj_paths = glob('data/000.5pc/*.npy')
# X = [np.load(traj_path) for traj_path in traj_paths]

xmin, xmax = -1.2, 1.2
tau = 25
k_opt=6
m_opt = 200

data_pc = '001.9pc'

#                   ('cluster',NDGrid(min=xmin, max=xmax, n_bins_per_feature=200)),
#                   ('msm', MaximumLikelihoodMSM(lag=1, score_method='vamp1'))])


# pickle.dump(model, open('model_lag1.pickl', 'wb'))

/Users/robert_arbon/anaconda/envs/ml4dyn/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/robert_arbon/anaconda/envs/ml4dyn/lib/python3.5/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [3]:
X = [np.load(x) for x in glob('data/'+data_pc+'/*npy')]

In [4]:
m_opt = int(np.sqrt(len(X)*X[0].shape[0]))
cluster = NDGrid(min=xmin, max=xmax, n_bins_per_feature=m_opt)
dtrajs = cluster.fit_transform(X)

In [5]:
train = dtrajs[:50]
test = dtrajs[50:]

In [6]:
msminit = MaximumLikelihoodMSM(lag=25)
msminit.fit(train)

MaximumLikelihoodMSM(connectivity='largest', count_mode='sliding',
           dt_traj='1 step', lag=25, maxerr=1e-08, maxiter=1000000,
           mincount_connectivity='1/n', reversible=True, score_k=10,
           score_method='VAMP2', sparse=False, statdist_constraint=None)

In [7]:
len(X)*(X[0].shape[0]-25)

156100

In [8]:
M = MaximumLikelihoodHMSM(lag=25, nstates=4, msm_init=msminit)
M.fit(train)
obs_prob = M.observation_probabilities
T = M.transition_matrix
pi = M.pi

(4, 4)
[[1.54610916e+04 1.04851116e+03 0.00000000e+00 2.14258303e-59]
 [9.28080694e+02 1.15207211e+04 8.15007130e-61 4.50176260e+02]
 [0.00000000e+00 1.39677570e-61 2.02686197e+04 3.40012603e+03]
 [8.83489351e-62 3.35657943e+02 3.56366282e+03 2.10733526e+04]]


In [9]:
np.sum(M.count_matrix)

78050.00000000006

In [10]:
ttrain = msminit.dtrajs_active
ttest = []
for x in test:
    try:
        ttest.append(msminit._full2active[x])
    except:
        pass
print(len(ttest))

49


In [11]:
loglik = 0
for obs in ttest:
    p_obs = obs_prob[:, obs].T
    loglik += forward(T, p_obs, pi)[0]
    
print(loglik)

-348470.55853994156


In [12]:
# # def forward(A, pobs, pi, T=None, alpha_out=None):
# """Compute P( obs | A, B, pi ) and all forward coefficients.

# Parameters
# ----------
# A : ndarray((N,N), dtype = float)
#     transition matrix of the hidden states
# pobs : ndarray((T,N), dtype = float)
#     pobs[t,i] is the observation probability for observation at time t given hidden state i
# pi : ndarray((N), dtype = float)
#     initial distribution of hidden states
# T : int, optional, default = None
#     trajectory length. If not given, T = pobs.shape[0] will be used.
# alpha_out : ndarray((T,N), dtype = float), optional, default = None
#     containter for the alpha result variables. If None, a new container will be created.

# Returns
# -------
# logprob : float
#     The probability to observe the sequence `ob` with the model given
#     by `A`, `B` and `pi`.
# alpha : ndarray((T,N), dtype = float), optional, default = None
#     alpha[t,i] is the ith forward coefficient of time t. These can be
#     used in many different algorithms related to HMMs.

# """

In [13]:
from sklearn.model_selection import ShuffleSplit
cv = ShuffleSplit(n_splits=20)

In [14]:
ks = np.arange(2,6)
k = 2
scores = np.zeros((ks.shape[0], cv.n_splits))
scores

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.]])

In [15]:
for kdx, k in enumerate(ks):
    print(k)
    for idx, (train_idx, test_idx) in enumerate(cv.split(dtrajs)):
        train = [dtrajs[i] for i in train_idx]
        train = [dtrajs[i] for i in test_idx]
        print('\t', idx)
        # initialize MInit
        Minit = MaximumLikelihoodMSM(lag=25)
        Minit.fit(train)

        # Map new trajectories
        ttrain = Minit.dtrajs_active
        ttest = []
        for x in test:
            try:
                ttest.append(Minit._full2active[x])
            except:
                pass

        # Fit HMM
        M = MaximumLikelihoodHMSM(lag=25, nstates=k, msm_init=Minit)
        M.fit(train)
        obs_prob = M.observation_probabilities
        T = M.transition_matrix
        pi = M.pi

        # Get log likelihood 
        loglik = 0
        for obs in ttest:
            p_obs = obs_prob[:, obs].T
            loglik += forward(T, p_obs, pi)[0]

        scores[kdx][idx] = loglik

2
	 0


KeyboardInterrupt: 

In [ ]:
scores_med = np.median(scores, axis=1)
scores_std = np.std(scores, axis=1)
data = {'k':ks, 'll_med':scores_med, 'll_err': scores_std}

In [ ]:
import pandas as pd
df = pd.DataFrame(data)

In [ ]:
plt.errorbar(df['k'],df['ll_med'])
plt.xlim((1,6))
# plt.ylim

In [ ]:
scores